In [1]:
import sys
from urllib.request import urlopen
import numpy as np
import pandas as pd

In [2]:
heartDisease_df = pd.read_csv('heart1.csv')

In [3]:
heartDisease_df.drop(['ca', 'slope', 'thal', 'oldpeak'], axis=1, inplace=True)

In [4]:
heartDisease_df.replace('?', np.nan, inplace=True)

In [5]:
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.models import BayesianModel
model = BayesianModel([('age', 'trestbps'),('age', 'fbs'), ('sex', 'trestbps'),('exang', 'trestbps'), ('trestbps', 'heartdisease'),('fbs', 'heartdisease'), ('heartdisease', 'restecg'),('heartdisease', 'thalach'), ('heartdisease', 'chol')])
# Learing CPDs using Maximum Likelihood Estimators
model.fit(heartDisease_df, estimator=MaximumLikelihoodEstimator)

In [6]:
print(model.get_cpds('age'))
print(model.get_cpds('chol'))
print(model.get_cpds('sex'))
model.get_independencies()

+---------+------------+
| age(29) | 0.00330033 |
+---------+------------+
| age(34) | 0.00660066 |
+---------+------------+
| age(35) | 0.0132013  |
+---------+------------+
| age(37) | 0.00660066 |
+---------+------------+
| age(38) | 0.00990099 |
+---------+------------+
| age(39) | 0.0132013  |
+---------+------------+
| age(40) | 0.00990099 |
+---------+------------+
| age(41) | 0.0330033  |
+---------+------------+
| age(42) | 0.0264026  |
+---------+------------+
| age(43) | 0.0264026  |
+---------+------------+
| age(44) | 0.0363036  |
+---------+------------+
| age(45) | 0.0264026  |
+---------+------------+
| age(46) | 0.0231023  |
+---------+------------+
| age(47) | 0.0165017  |
+---------+------------+
| age(48) | 0.0231023  |
+---------+------------+
| age(49) | 0.0165017  |
+---------+------------+
| age(50) | 0.0231023  |
+---------+------------+
| age(51) | 0.039604   |
+---------+------------+
| age(52) | 0.0429043  |
+---------+------------+
| age(53) | 0.0264026  |


(age _|_ sex, exang)
(age _|_ sex, exang | fbs)
(age _|_ chol, thalach, restecg | heartdisease)
(age _|_ exang | sex)
(age _|_ sex | exang)
(age _|_ chol, restecg | heartdisease, thalach)
(age _|_ chol, heartdisease, thalach, restecg | fbs, trestbps)
(age _|_ chol, thalach, restecg | heartdisease, trestbps)
(age _|_ chol, thalach | heartdisease, restecg)
(age _|_ thalach, restecg | chol, heartdisease)
(age _|_ chol, thalach, restecg | fbs, heartdisease)
(age _|_ exang | fbs, sex)
(age _|_ sex | fbs, exang)
(age _|_ chol, thalach, restecg | sex, heartdisease)
(age _|_ chol, thalach, restecg | heartdisease, exang)
(age _|_ chol, heartdisease, restecg | fbs, thalach, trestbps)
(age _|_ chol, restecg | heartdisease, thalach, trestbps)
(age _|_ chol | heartdisease, thalach, restecg)
(age _|_ restecg | chol, heartdisease, thalach)
(age _|_ chol, restecg | heartdisease, fbs, thalach)
(age _|_ chol, restecg | heartdisease, sex, thalach)
(age _|_ chol, restecg | heartdisease, thalach, exang)
(a

In [7]:
from pgmpy.inference import VariableElimination
HeartDisease_infer = VariableElimination(model)

In [8]:
q = HeartDisease_infer.query(variables=['heartdisease'], evidence={'age': 28})
# prob_offer = olympic_infer.query(variables = ['Offer'])
# print(q['heartdisease'])
print(q)

F:\software\Anaconda\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:518: UserWarning: Found unknown state name. Trying to switch to using all state names as state numbers
  warn(
Eliminating: exang: 100%|███████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 101.47it/s]

+-----------------+---------------------+
| heartdisease    |   phi(heartdisease) |
+=================+=====================+
| heartdisease(0) |              0.4058 |
+-----------------+---------------------+
| heartdisease(1) |              0.5942 |
+-----------------+---------------------+


In [9]:
q = HeartDisease_infer.query(variables=['heartdisease'],evidence={'chol': 100})
print(q)

Finding Elimination Order: : 100%|██████████████████████████████████████████████████████| 7/7 [00:00<00:00, 269.19it/s]

Eliminating: exang: 100%|████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 60.87it/s]

+-----------------+---------------------+
| heartdisease    |   phi(heartdisease) |
+=================+=====================+
| heartdisease(0) |              0.0000 |
+-----------------+---------------------+
| heartdisease(1) |              1.0000 |
+-----------------+---------------------+
